<a href="https://colab.research.google.com/github/Galmieux/AIGakki/blob/main/%E3%82%B9%E3%82%AF%E3%83%AC%E3%82%A4%E3%83%94%E3%83%B3%E3%82%B0%E5%AE%8C%E6%88%90%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import requests
from bs4 import BeautifulSoup
import os
import time

# 基本URLとページURLのテンプレート
base_url = "https://www.jstage.jst.go.jp"
url_template = "https://www.jstage.jst.go.jp/browse/pjsai/JSAI2024/0/_contents/-char/ja?from={}"

# PDFを保存するディレクトリを作成
pdf_dir = "jstage_pdfs"#/content/drive/MyDrive/色々/jstage_pdfsに修正？？
os.makedirs(pdf_dir, exist_ok=True)

# from=0からfrom=18までページを処理
for i in range(0, 19):
    url = url_template.format(i)  # 各ページのURLを生成
    print(f"処理中のURL: {url}")

    # ページへのリクエストを送信
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # "_pdf"を含むPDFリンクを見つける
    pdf_links = soup.find_all("a", href=lambda href: href and "_pdf" in href)

    # 各PDFをダウンロード
    for index, link in enumerate(pdf_links):
        pdf_url = link["href"]

        # 相対パスをフルパスに変換
        if not pdf_url.startswith("http"):
            pdf_url = base_url + pdf_url

        # ファイル名をリンクの最後の部分から取得し、ディレクトリに保存
        pdf_name = os.path.join(pdf_dir, f"document_{i}_{index}.pdf")

        try:
            # PDFをダウンロード
            pdf_response = requests.get(pdf_url, timeout=30)
            pdf_response.raise_for_status()  # ステータスコードが200でない場合例外を発生させる

            # ダウンロードしたコンテンツが正しいか確認
            if 'application/pdf' in pdf_response.headers['Content-Type']:
                # PDFを保存
                with open(pdf_name, "wb") as pdf_file:
                    pdf_file.write(pdf_response.content)
                print(f"ダウンロード成功: {pdf_name}")
            else:
                print(f"無効なコンテンツタイプ: {pdf_url} - {pdf_response.headers['Content-Type']}")

        except requests.exceptions.RequestException as e:
            print(f"ダウンロード失敗: {pdf_url}, エラー: {e}")

        # リクエスト間に待機時間を追加
        time.sleep(1)  # 1秒待機

print("ダウンロードしたPDF:", os.listdir(pdf_dir))


処理中のURL: https://www.jstage.jst.go.jp/browse/pjsai/JSAI2024/0/_contents/-char/ja?from=0


KeyboardInterrupt: 